In [39]:
import torch
import torch.nn as nn

import torch.optim as optim

from torch.utils.data import TensorDataset, DataLoader, DistributedSampler

device = 'cuda' if torch.cuda.is_available() else 'cpu'

X = torch.rand(8192, 3).to(device)
y_transform = lambda x: (x[:, 0]**3)*3 + 4*(x[:, 1]**2) + (4*x[:, 2])
y = y_transform(X)

batch_size = 128
dataset = TensorDataset(X, y)
dataLoader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

class myModel(nn.Module):
    def __init__(self, input_dims):
        super().__init__()
        self.dims = 8192
        self.model = nn.Sequential(
            nn.Linear(input_dims, self.dims),
            nn.ReLU(),
            nn.Linear(self.dims, self.dims),
            nn.ReLU(),
            nn.Linear(self.dims, self.dims),
            nn.ReLU(),            
            nn.Linear(self.dims, 1)
        )
        
    def forward(self, x):
        x = self.model(x)
        return x
        
model = myModel(X.shape[1]).to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

epochs = 10

for epoch in range(epochs):
    
    count = 0
    
    for X_batch, y_batch in dataLoader:
    
        optimizer.zero_grad()

        y_pred = model(X_batch)

        loss = loss_fn(y_pred, y_batch.view(batch_size, -1))
        
        if count == 0:
            print(f'good batch = {y_batch.view(batch_size, -1)}')
            print(f'bad batch = {y_batch.view(-1)}')

        loss.backward()
        optimizer.step()
        
    if epoch % (epochs / 10) == 0:
        print(loss)    

print()
    
print(y_transform(torch.tensor([[.3,.4,.5]]).to(device)))
print(model(torch.tensor([.3,.4,.5]).to(device)))

tensor(3.4977, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(3.7404, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(3.6001, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(2.9250, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(4.0509, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(3.1698, device='cuda:0', grad_fn=<MseLossBackward0>)


KeyboardInterrupt: 